In [1]:
# === Imports ===
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dropout, GlobalMaxPooling1D, Dense
from tensorflow.keras.optimizers import Adam

# === Load and preprocess the dataset ===
df = pd.read_excel("default of credit card clients.xls", skiprows=1)
df.rename(columns={"default payment next month": "target"}, inplace=True)
df.drop(columns=['ID'], inplace=True)

# Clean and scale
X = df.drop(columns=['target']).values.astype(np.float32)
y = df['target'].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Train-test split and reshape for BDLSTM
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train_seq = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_seq = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# === Build Functional BDLSTM model ===
inputs = Input(shape=(1, X_train.shape[1]))
x = Bidirectional(LSTM(60, return_sequences=True))(inputs)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(80, return_sequences=True))(x)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(120, return_sequences=True))(x)
x = Dropout(0.2)(x)
x = GlobalMaxPooling1D()(x)
outputs = Dense(1, activation='sigmoid')(x)

bdlstm_model = Model(inputs, outputs)
bdlstm_model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# === Train BDLSTM ===
bdlstm_model.fit(X_train_seq, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

# === Feature Extraction ===
feature_extractor = Model(inputs=bdlstm_model.input, outputs=x)
X_train_feat = feature_extractor.predict(X_train_seq)
X_test_feat = feature_extractor.predict(X_test_seq)

# === XGBoost on Extracted Features ===
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_feat, y_train)

# === Evaluate ===
y_pred = xgb.predict(X_test_feat)
y_prob = xgb.predict_proba(X_test_feat)[:, 1]

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("📈 F1 Score:", f1_score(y_test, y_pred))
print("🏆 ROC AUC:", roc_auc_score(y_test, y_prob))
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))


Epoch 1/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.7995 - loss: 0.4927 - val_accuracy: 0.8092 - val_loss: 0.4615
Epoch 2/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8128 - loss: 0.4531 - val_accuracy: 0.8125 - val_loss: 0.4557
Epoch 3/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8179 - loss: 0.4454 - val_accuracy: 0.8112 - val_loss: 0.4535
Epoch 4/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8188 - loss: 0.4442 - val_accuracy: 0.8140 - val_loss: 0.4599
Epoch 5/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8258 - loss: 0.4321 - val_accuracy: 0.8077 - val_loss: 0.4537
Epoch 6/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8168 - loss: 0.4424 - val_accuracy: 0.8135 - val_loss: 0.4569
Epoch 7/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8164 - loss: 0.4443 - val_accuracy: 0.8142 - val_loss: 0.4506
Epoch 8/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8211 - loss: 0.4344 - val_accuracy: 0

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:46:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Accuracy: 0.8073333333333333
📈 F1 Score: 0.4618249534450652
🏆 ROC AUC: 0.7432974078187462

📄 Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.93      0.88      4673
           1       0.60      0.37      0.46      1327

    accuracy                           0.81      6000
   macro avg       0.72      0.65      0.67      6000
weighted avg       0.79      0.81      0.79      6000

